In [1]:
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

from vecstack import stacking

import optuna

In [2]:
train = pd.read_csv('preprocessed_train.csv', encoding='euc-kr')
test = pd.read_csv('preprocessed_test.csv', encoding='euc-kr')

In [3]:
train

,분석데이터,label,numstrings,avlength,printables,entropy,paths,urls,registry,MZ,...,dist_86,dist_87,dist_88,dist_89,dist_90,dist_91,dist_92,dist_93,dist_94,dist_95
0,1,1,144,12.298611,1771,5.356616,0,0,0,1,...,10,4,10,9,4,0,1,0,0,0
1,2,1,804,9.580846,7703,6.063542,0,0,0,6,...,43,121,84,78,47,36,40,45,27,36
2,3,0,2205,12.736054,28083,6.107050,9,0,0,6,...,326,268,239,286,199,148,154,37,48,36
3,4,0,2602,10.288240,26770,5.373013,8,0,0,1,...,336,230,206,245,76,0,26,702,1,5
4,5,1,8980,23.252339,208806,5.775223,0,28,16,3,...,731,882,1171,1010,322,64,327,84,75,244
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,1,2018,13.938057,28127,5.940442,0,70,0,11,...,246,186,206,235,88,33,81,58,61,72
9996,9997,0,1105,16.437104,18163,5.766962,0,11,0,3,...,199,57,134,123,20,25,28,25,41,13
9997,9998,0,4,58.500000,234,3.811827,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9998,9999,1,3312,24.939312,82599,5.834730,0,39,0,8,...,438,985,806,851,113,123,181,100,75,86


In [4]:
x_train = train.drop(columns=['분석데이터', 'label'], axis=1)

In [5]:
y_train = train['label']

In [6]:
x_test = test.drop(columns='분석대상', axis=1)

In [ ]:
# train_df_x = train.drop(['label'], axis=1)

In [ ]:
# train_df_y = train['label']

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(train_df_x, train_df_y, test_size=0.2, random_state=42)

In [ ]:
from xgboost import XGBClassifier

xgb_final = XGBClassifier(silent=False,
                              n_estimators = 10000,
                              booster='gbtree',
                              tree_method='gpu_hist',
                              preidctor= 'gpu_predictor',
                              scale_pos_weight=1,
                              learning_rate=0.03689407512484644,
                              objective='binary:logistic',
                              max_depth = 8,
                              subsample = 0.780714581166012,
                              colsample_bytree = 0.3723914688159835,
                              gamma = 0,
                              reg_lambda = 50.0,
                              random_state=42)

In [ ]:
final_pred = xgb_final.fit(x_train, y_train).predict(x_test)

In [ ]:
accuracy_score(y_test, final_pred)

In [ ]:
confusion_matrix(y_test, final_pred)

In [ ]:
cb_model = CatBoostClassifier(objective = 'CrossEntropy',
                              depth = 12,
                              boosting_type = 'Ordered',
                              bootstrap_type = 'Bernoulli',
                              learning_rate = 0.19719860541901787,
                              iterations = 204,
                              random_strength= 34,
                              od_type= 'IncToDec',
                              subsample=0.9558805603499683,
                              colsample_bylevel = 0.043529438827711514
                             )

In [ ]:
cb_pred = cb_model.fit(x_train, y_train).predict(x_test)

In [ ]:
accuracy_score(y_test, cb_pred)

In [ ]:
confusion_matrix(y_test, cb_pred)

In [ ]:
"""
lgbm_model = LGBMClassifier(n_estimators = 10000,
                            learning_rate = 0.09416659111369403,
                            max_depth = 43,
                            boosting = 'gbdt',
                            objective = 'binary',
                            metric = 'binary_logloss',
                            is_training_metric = True,
                            num_leaves = 41,
                            min_data_in_leaf = 10,
                            feature_fraction = 0.8,
                            bagging_fraction = 0.9,
                            bagging_freq = 0,
                            alpha = 0.019782149081578264)
"""

In [ ]:
# lgbm_pred = lgbm_model.fit(x_train, y_train).predict(x_test)

In [ ]:
# accuracy_score(y_test, lgbm_pred)

In [ ]:
# confusion_matrix(y_test, lgbm_pred)

In [7]:
xgb_params = {'n_estimators': 10000,
               'learning_rate': 0.03689407512484644,
               'max_depth': 8,
               'colsample_bytree': 0.3723914688159835,
               'subsample': 0.780714581166012,
               'eval_metric': 'auc',
               'use_label_encoder': False,
               'gamma': 0,
               'reg_lambda': 50.0,
               'tree_method': 'gpu_hist',
               'gpu_id': 0,
               'predictor': 'gpu_predictor',
               'random_state': 42 }

lgb_params = {'n_estimators': 10000,
              'learning_rate':0.09416659111369403,
              'max_depth':43,
              'boosting':'gbdt',
              'objective': 'binary',
              'metric': 'binary_logloss',
              'is_training_metric': True,
              'num_leaves':41,
              'min_data_in_leaf':10,
              'feature_fraction':0.8,
              'bagging_fraction':0.9,
              'bagging_freq':0,
              'alpha': 0.019782149081578264 }

cat_params = {'objective': 'CrossEntropy',
              'colsample_bylevel': 0.043529438827711514,
              'depth': 12,
              'boosting_type': 'Ordered',
              'bootstrap_type': 'Bernoulli',
              'learning_rate': 0.19719860541901787,
              'iterations': 205,
              'random_strength': 34,
              'od_type': 'IncToDec',
              'subsample': 0.9558805603499683
             }

In [8]:
lgbm = LGBMClassifier(**lgb_params)

xgb = XGBClassifier(**xgb_params)

cat = CatBoostClassifier(**cat_params)

In [9]:
models = [lgbm, xgb, cat]

In [10]:
S_train, S_test = stacking(models, x_train, y_train, x_test, regression=False,
                          metric=accuracy_score, n_folds=5, stratified=True, shuffle=True, random_state=42, verbose=2)

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LGBMClassifier]
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0
    fold  0:  [0.93200000]
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[Li

90:	learn: 0.1124196	total: 4m 25s	remaining: 5m 32s
91:	learn: 0.1112003	total: 4m 28s	remaining: 5m 29s
92:	learn: 0.1103739	total: 4m 31s	remaining: 5m 26s
93:	learn: 0.1102490	total: 4m 34s	remaining: 5m 24s
94:	learn: 0.1076705	total: 4m 37s	remaining: 5m 21s
95:	learn: 0.1073107	total: 4m 40s	remaining: 5m 18s
96:	learn: 0.1070646	total: 4m 43s	remaining: 5m 15s
97:	learn: 0.1063562	total: 4m 46s	remaining: 5m 12s
98:	learn: 0.1062419	total: 4m 49s	remaining: 5m 9s
99:	learn: 0.1059645	total: 4m 51s	remaining: 5m 6s
100:	learn: 0.1032969	total: 4m 55s	remaining: 5m 3s
101:	learn: 0.1031856	total: 4m 57s	remaining: 5m
102:	learn: 0.1026565	total: 5m	remaining: 4m 57s
103:	learn: 0.1023048	total: 5m 2s	remaining: 4m 54s
104:	learn: 0.1020251	total: 5m 5s	remaining: 4m 50s
105:	learn: 0.1009242	total: 5m 8s	remaining: 4m 47s
106:	learn: 0.0986652	total: 5m 11s	remaining: 4m 45s
107:	learn: 0.0982484	total: 5m 14s	remaining: 4m 42s
108:	learn: 0.0953967	total: 5m 17s	remaining: 4m 39

40:	learn: 0.2325470	total: 1m 58s	remaining: 7m 53s
41:	learn: 0.2309791	total: 2m	remaining: 7m 49s
42:	learn: 0.2255508	total: 2m 4s	remaining: 7m 48s
43:	learn: 0.2237809	total: 2m 6s	remaining: 7m 44s
44:	learn: 0.2211900	total: 2m 9s	remaining: 7m 40s
45:	learn: 0.2185222	total: 2m 12s	remaining: 7m 39s
46:	learn: 0.2144645	total: 2m 15s	remaining: 7m 36s
47:	learn: 0.2117780	total: 2m 18s	remaining: 7m 33s
48:	learn: 0.2053936	total: 2m 21s	remaining: 7m 31s
49:	learn: 0.2027709	total: 2m 24s	remaining: 7m 28s
50:	learn: 0.1997835	total: 2m 27s	remaining: 7m 25s
51:	learn: 0.1962588	total: 2m 30s	remaining: 7m 23s
52:	learn: 0.1930068	total: 2m 33s	remaining: 7m 19s
53:	learn: 0.1907136	total: 2m 35s	remaining: 7m 15s
54:	learn: 0.1875175	total: 2m 38s	remaining: 7m 12s
55:	learn: 0.1838662	total: 2m 41s	remaining: 7m 10s
56:	learn: 0.1806145	total: 2m 44s	remaining: 7m 7s
57:	learn: 0.1767125	total: 2m 48s	remaining: 7m 6s
58:	learn: 0.1751544	total: 2m 51s	remaining: 7m 3s
59:

195:	learn: 0.0682336	total: 9m 40s	remaining: 26.7s
196:	learn: 0.0681625	total: 9m 43s	remaining: 23.7s
197:	learn: 0.0681601	total: 9m 44s	remaining: 20.7s
198:	learn: 0.0681216	total: 9m 47s	remaining: 17.7s
199:	learn: 0.0680920	total: 9m 50s	remaining: 14.8s
200:	learn: 0.0673097	total: 9m 53s	remaining: 11.8s
201:	learn: 0.0672889	total: 9m 55s	remaining: 8.85s
202:	learn: 0.0672139	total: 9m 58s	remaining: 5.9s
203:	learn: 0.0668184	total: 10m 1s	remaining: 2.95s
204:	learn: 0.0660092	total: 10m 5s	remaining: 0us
    fold  1:  [0.89100000]
0:	learn: 0.6365806	total: 2.62s	remaining: 8m 54s
1:	learn: 0.5963916	total: 5.91s	remaining: 9m 59s
2:	learn: 0.5544152	total: 8.6s	remaining: 9m 39s
3:	learn: 0.5247699	total: 11.1s	remaining: 9m 18s
4:	learn: 0.5002503	total: 14.5s	remaining: 9m 38s
5:	learn: 0.4863411	total: 17.3s	remaining: 9m 34s
6:	learn: 0.4695570	total: 20.2s	remaining: 9m 31s
7:	learn: 0.4517139	total: 22.7s	remaining: 9m 19s
8:	learn: 0.4298498	total: 25.2s	remain

145:	learn: 0.0752650	total: 7m 8s	remaining: 2m 53s
146:	learn: 0.0740123	total: 7m 11s	remaining: 2m 50s
147:	learn: 0.0738330	total: 7m 14s	remaining: 2m 47s
148:	learn: 0.0734650	total: 7m 17s	remaining: 2m 44s
149:	learn: 0.0729617	total: 7m 20s	remaining: 2m 41s
150:	learn: 0.0728142	total: 7m 23s	remaining: 2m 38s
151:	learn: 0.0708856	total: 7m 26s	remaining: 2m 35s
152:	learn: 0.0708540	total: 7m 29s	remaining: 2m 32s
153:	learn: 0.0699379	total: 7m 32s	remaining: 2m 29s
154:	learn: 0.0698888	total: 7m 35s	remaining: 2m 26s
155:	learn: 0.0698170	total: 7m 38s	remaining: 2m 24s
156:	learn: 0.0689279	total: 7m 41s	remaining: 2m 20s
157:	learn: 0.0688963	total: 7m 43s	remaining: 2m 17s
158:	learn: 0.0687865	total: 7m 47s	remaining: 2m 15s
159:	learn: 0.0674869	total: 7m 50s	remaining: 2m 12s
160:	learn: 0.0656490	total: 7m 53s	remaining: 2m 9s
161:	learn: 0.0656008	total: 7m 56s	remaining: 2m 6s
162:	learn: 0.0655430	total: 7m 58s	remaining: 2m 3s
163:	learn: 0.0653666	total: 8m 

95:	learn: 0.1110240	total: 4m 49s	remaining: 5m 28s
96:	learn: 0.1107351	total: 4m 52s	remaining: 5m 25s
97:	learn: 0.1096501	total: 4m 55s	remaining: 5m 22s
98:	learn: 0.1093255	total: 4m 58s	remaining: 5m 19s
99:	learn: 0.1057709	total: 5m	remaining: 5m 15s
100:	learn: 0.1053289	total: 5m 4s	remaining: 5m 13s
101:	learn: 0.1049367	total: 5m 7s	remaining: 5m 10s
102:	learn: 0.1044657	total: 5m 9s	remaining: 5m 6s
103:	learn: 0.1034016	total: 5m 12s	remaining: 5m 3s
104:	learn: 0.1032905	total: 5m 15s	remaining: 5m
105:	learn: 0.1031654	total: 5m 18s	remaining: 4m 57s
106:	learn: 0.1030910	total: 5m 21s	remaining: 4m 54s
107:	learn: 0.1018329	total: 5m 25s	remaining: 4m 51s
108:	learn: 0.1018088	total: 5m 28s	remaining: 4m 48s
109:	learn: 0.1011756	total: 5m 30s	remaining: 4m 45s
110:	learn: 0.1003886	total: 5m 33s	remaining: 4m 42s
111:	learn: 0.1001279	total: 5m 36s	remaining: 4m 39s
112:	learn: 0.0999779	total: 5m 38s	remaining: 4m 35s
113:	learn: 0.0975248	total: 5m 42s	remaining:

45:	learn: 0.2165606	total: 2m 14s	remaining: 7m 46s
46:	learn: 0.2131292	total: 2m 17s	remaining: 7m 43s
47:	learn: 0.2111664	total: 2m 20s	remaining: 7m 39s
48:	learn: 0.2044668	total: 2m 23s	remaining: 7m 37s
49:	learn: 0.2018673	total: 2m 26s	remaining: 7m 34s
50:	learn: 0.1979536	total: 2m 29s	remaining: 7m 31s
51:	learn: 0.1947327	total: 2m 32s	remaining: 7m 28s
52:	learn: 0.1922240	total: 2m 35s	remaining: 7m 26s
53:	learn: 0.1904668	total: 2m 37s	remaining: 7m 21s
54:	learn: 0.1868247	total: 2m 41s	remaining: 7m 19s
55:	learn: 0.1839404	total: 2m 44s	remaining: 7m 16s
56:	learn: 0.1788059	total: 2m 47s	remaining: 7m 13s
57:	learn: 0.1771225	total: 2m 50s	remaining: 7m 12s
58:	learn: 0.1747137	total: 2m 53s	remaining: 7m 9s
59:	learn: 0.1726810	total: 2m 56s	remaining: 7m 6s
60:	learn: 0.1700629	total: 2m 59s	remaining: 7m 4s
61:	learn: 0.1682165	total: 3m 2s	remaining: 7m 1s
62:	learn: 0.1660431	total: 3m 5s	remaining: 6m 58s
63:	learn: 0.1610381	total: 3m 8s	remaining: 6m 54s


199:	learn: 0.0676989	total: 10m 8s	remaining: 15.2s
200:	learn: 0.0675431	total: 10m 11s	remaining: 12.2s
201:	learn: 0.0675289	total: 10m 14s	remaining: 9.12s
202:	learn: 0.0675288	total: 10m 17s	remaining: 6.08s
203:	learn: 0.0672728	total: 10m 20s	remaining: 3.04s
204:	learn: 0.0655140	total: 10m 23s	remaining: 0us
    fold  4:  [0.90750000]
    ----
    MEAN:     [0.89950000] + [0.00731437]
    FULL:     [0.89950000]



In [11]:
model = lgbm

In [12]:
model = model.fit(S_train, y_train)

[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=10, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=10
[LightGBM] [Warning] bagging_fraction is set=0.9, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9
[LightGBM] [Warning] bagging_freq is set=0, subsample_freq=0 will be ignored. Current value: bagging_freq=0


In [13]:
y_pred = model.predict(S_test)

In [ ]:
# print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

In [ ]:
# confusion_matrix(y_test, y_pred)

In [ ]:
y_pred

In [ ]:
y_pred.shape

In [ ]:
"""
sub = pd.read_csv('submission_ex.csv', encoding='euc-kr')
sub['정답지'] = y_pred
sub.to_csv('sub_stack.csv', index = 0, encoding='euc-kr')
"""

## sub_stack, sub_rf 비교

In [ ]:
compare = pd.DataFrame(columns=['status'])

In [ ]:
compare

In [ ]:
sub_stack = pd.read_csv('sub_stack.csv', encoding='euc-kr')

In [ ]:
sub_rf = pd.read_csv('rf_sub.csv', encoding='euc-kr')

In [ ]:
sub_stack

In [ ]:
sub_rf

In [ ]:
for i in range(5000):
    if sub_rf['정답지'][i] == sub_stack['정답지'][i]:
        compare.loc[i] = 0
    else:
        compare.loc[i] = 1

In [ ]:
(compare['status']==0).sum()

In [ ]:
(compare['status']==1).sum()

## Soft Voting (RF + Stacking Ensemble + XGBoost, LGBM, CatBoost)

1. 각각의 결과 csv파일들을 비교해보기  
얼마나 다른지를 알아보기 위해서 결과 csv파일들을 비교해 봅니다.